<a href="https://colab.research.google.com/github/devloper13/SiameseNetworkProject/blob/master/Siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from builtins import input

#import system things
from tensorflow.examples.tutorials.mnist import input_data # for data
import tensorflow as tf
import numpy as np
import os
import keras


# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
import tensorflow as tf
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
import pickle
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [50]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.util import ngrams
import nltk
nltk.download('stopwords')

nltk.download('punkt')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from tqdm import tqdm
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
class Siamese:

    # Create model
    def __init__(self,dname="Siamese"):
        self.x1 = tf.placeholder(tf.float32, [None, 64881])
        self.x2 = tf.placeholder(tf.float32, [None, 64881])

        with tf.variable_scope(dname) as scope:
            self.o1 = self.network(self.x1)
            scope.reuse_variables()
            self.o2 = self.network(self.x2)

        # Create loss
        self.y_ = tf.placeholder(tf.float32, [None])
        #self.y_ = tf.placeholder(tf.int32, [None])
        self.loss = self.cosineLoss()

    def network(self, x):
        
        x = tf.reshape(x,shape=[-1,1,64881,1])
        activated_conv1 = self.conv_layer('conv_1',x,3)
        maxpool1 = self.maxpool_layer('maxp_1',activated_conv1)
        
        flattened_conv = tf.layers.flatten(maxpool1)   #To be removed
        activated_fc1 = self.fc_layer( "fc1",flattened_conv, 128)
        #activated_fc2 = self.fc_layer("fc2",activated_fc1, 1024)
        #activated_fc3 = self.fc_layer("fc3",activated_fc2, 2)
        
        return activated_fc1
        
    def conv_layer(self,name,inputs,cur_channel):
        #print(inputs.get_shape())
        prev_channel = inputs.get_shape()[-1]
        #print(prev_channel)
        init = tf.variance_scaling_initializer(scale=2.0)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[1,10,prev_channel,cur_channel],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_channel],initializer = init)
        conv = tf.nn.conv2d(inputs,w,strides=[1,1,1,1],padding = "SAME")
        activation = conv+b
        return activation  
      
    def maxpool_layer(self,name,inputs):
        return tf.nn.relu(tf.nn.max_pool(inputs,ksize=[1,1,100,1],strides=[1,1,100,1],padding="SAME"))  
    
    def fc_layer(self,name,inputs,cur_layer):
        print(inputs.get_shape())
        prev_layer = inputs.get_shape()[-1]
        init = tf.truncated_normal_initializer(stddev=0.01)
        w = tf.get_variable(name+"_w",dtype=tf.float32,shape=[prev_layer,cur_layer],initializer=init)
        b = tf.get_variable(name+"_b",dtype=tf.float32,shape=[cur_layer],initializer=init)
        activation = tf.matmul(inputs,w)+b
        return activation
    
        
    def cosineLoss(self):
        
        norms1 = tf.norm(self.o1,axis=1)
        norms2 = tf.norm(self.o2,axis=1)
        norm = tf.multiply(norms1,norms2)
        cosines = tf.div(tf.reduce_sum(tf.multiply(self.o1,self.o2),axis=1),norm)
        
        labels_t = self.y_
        labels_f = tf.subtract(1.0, self.y_, name="1-yi")          # labels_ = !labels;
        
        
        C = tf.constant(0.85, name="C")
        
        pos = tf.multiply(labels_t,tf.subtract(1.0,cosines), name="yi_x_cosine")
        
        neg = tf.multiply(labels_f, tf.maximum(tf.subtract(cosines,C),0), name="Nyi_x_C-cosine")
        losses = tf.add(pos, neg, name="losses")
        loss = tf.reduce_mean(losses, name="loss")
        return loss
        

          

In [83]:
link="https://drive.google.com/open?id=1Jhj9OazxPnvLcuuZsZvNFfpnnsFg88I7"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('three_hash')  
with open('three_hash','rb') as f:
   three_hash_dict = pickle.load(f)
print(len(three_hash_dict))

30628


In [0]:
stop_words = set(stopwords.words('english')) 
def getThreeHash(text):
  vectorizer = CountVectorizer()
  tokenizer=vectorizer.build_tokenizer()
  
  hashes=""
  tokens=tokenizer(text)
  ps = PorterStemmer()
  tokens = [ps.stem(word) for word in tokens]
  for token in tokens:
    if token not in stop_words:
      tokenModi="#"+token+"#"
      output = list(ngrams(tokenModi, 3))
      for a in output:
        hashes+=(''.join(a))+" "
  
  return(hashes)
# quest = "where do i find hot girls"
# hash_var = getThreeHash(quest)
#print(hash_var)

In [85]:
link="https://drive.google.com/open?id=17VMN5CJA05vTPEs15gw-W2ocxmEITQEH" #create shareable link of google drive file
fluff, id = link.split('=')
print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('QA.csv')  
ndata = pd.read_csv('QA.csv',error_bad_lines=False).values


17VMN5CJA05vTPEs15gw-W2ocxmEITQEH


In [0]:
data = ndata[:1000,1:]

In [0]:
def preprocess(text):
    tokens = word_tokenize(text)
    tokenlower = [word.lower() for word in tokens]
    
    stopWords = set(stopwords.words('english'))
    tokenlower = [word for word in stopWords if word not in stopWords]
    
    tokenDict = nltk.defaultdict(int)
    for word in tokens:
        tokenDict[word] += 1
    return tokenDict

def cosDistance(v1,v2):
    dotProduct = np.dot(v1,v2)
    normV1 = np.linalg.norm(v1)
    normV2 = np.linalg.norm(v2)
#     print(dotProduct,normV1,normV2)
    return dotProduct / (normV1 * normV2)

def getSimilarity(vDict1,vDict2):
    allWords = []
    for key in vDict1:
        allWords.append(key)
    for key in vDict2:
        allWords.append(key)
    allWordsSize = len(allWords)
    
    v1 = np.zeros(allWordsSize,dtype=np.int)
    v2 = np.zeros(allWordsSize,dtype=np.int)
    
    i = 0
    for key in allWords:
#         print(key)
        v1[i] = vDict1.get(key, 0)
        v2[i] = vDict2.get(key, 0)
        i += 1
#     print(v1,v2)    
    return cosDistance(v1,v2)

In [0]:
def vectorize(hashString,dictionary):
  
  vectorizer = CountVectorizer()
  tokenizer = vectorizer.build_tokenizer()
  vec = [0]*64881
  
  for token in tokenizer(hashString):
    try:
      vec[dictionary[token]] += 1
    except:
      pass
    
  return vec

def createBatch(data,goodSet,badSetQues,badSetAns,dictionary):
  
  good_data = data[goodSet]
  bad_quest = data[badSetQues,0]
  bad_anser = data[badSetAns,1]
  questions = np.concatenate((good_data[:,0],bad_quest))
  answers = np.concatenate((good_data[:,1],bad_anser))
  label = np.array([1]*good_data.shape[0] + [0]*bad_quest.shape[0])
  
  ques,ans = [],[]
  for d in questions:
    ques += [vectorize(getThreeHash(d.lower()),dictionary)]
  for i,d in enumerate(answers):
    ans += [vectorize(getThreeHash(str(d).lower()),dictionary)]
  return np.array(ques),np.array(ans),label
    
def getRandomBatch(data,batchsize,dictionary,good_bad=0.5):
  
  goodSet = np.random.permutation(data.shape[0])[:int(batchsize*good_bad)]
  badSetQues = np.random.permutation(data.shape[0])[:batchsize - int(batchsize*good_bad)]
  badSetAns = np.random.permutation(data.shape[0])[:batchsize - int(batchsize*good_bad)]
  return createBatch(data,goodSet,badSetQues,badSetAns,dictionary)
  

    

In [90]:
tf.reset_default_graph()
siamese = Siamese('siamese6')
try:
  sess.close()
except:
  pass
sess = tf.InteractiveSession()
train_Step = tf.train.MomentumOptimizer(0.01,0.05).minimize(siamese.loss)
tf.initialize_all_variables().run()

(?, 1947)
(?, 1947)


In [0]:
def trainSiameseNetwork(data,siamese,sess,batchSize,epochs,dictionary):
  
  #Siamese Network
  #sess = tf.Session()
  #tf.reset_default_graph()
  #siamese = Siamese()
  
  while epochs > 0:
    
    avg = 0.0
    permSet = np.random.permutation(data.shape[0])

    for p in range(0,permSet.shape[0],batchSize):

      goodSet = np.array(list(range(p,p+batchSize)))
      badSetQ = goodSet.copy()
      badSetA = np.array(permSet[p:p+batchSize])
      ques,ans,labl = createBatch(data,goodSet,badSetQ,badSetA,dictionary)


      #print(ques.shape,ans.shape,labl.shape)
      _, Loss = sess.run([train_Step,siamese.loss], feed_dict={
                    siamese.x1: ques,
                    siamese.x2: ans,
                    siamese.y_: labl})
      avg += Loss
      #print("Epoch",epochs,"Batch",p/batchSize,"Loss",Loss)
    
    print("Average Epoch ",epochs,"Loss ",avg/(data.shape[0]/batchSize))
    epochs -= 1
        

In [0]:
def predict(qset,ques,siamese,sess,dictionary,alpha=0.5,best=3):
  
  vques  = preprocess(ques)
  thQues = np.array(vectorize(getThreeHash(ques.lower()),dictionary)).reshape(1,-1)

  scores = []

  for q in qset:

    oQues = np.array(vectorize(getThreeHash(q.lower()),dictionary)).reshape(1,-1)
    vQues = preprocess(q)
    Loss = sess.run([siamese.loss],feed_dict={
                    siamese.x1: thQues,
                    siamese.x2: oQues,
                    siamese.y_: np.array([1.0])})
    tLoss = getSimilarity(vques,vQues)
    print(alpha*Loss,(1-alpha)alphatLoss)
    scores += [alpha*Loss[0] + (1-alpha)*(1-tLoss)]

  sscores = np.argsort(np.array(scores))[:best]
  return qset[sscores],scores
  

In [93]:

trainSiameseNetwork(data,siamese,sess,100,10,three_hash_dict)

Average Epoch  10 Loss  0.10446406602859497
Average Epoch  9 Loss  0.0641596931964159
Average Epoch  8 Loss  0.06335491687059402
Average Epoch  7 Loss  0.06259525753557682
Average Epoch  6 Loss  0.06240963861346245
Average Epoch  5 Loss  0.06196700893342495
Average Epoch  4 Loss  0.06205567419528961
Average Epoch  3 Loss  0.06167623475193977
Average Epoch  2 Loss  0.06178317740559578
Average Epoch  1 Loss  0.06152181327342987


In [0]:
trainQues = data[:,0]

In [119]:
ques = input("Enter a question ")

Enter a question what is a stork


In [124]:
score = predict(trainQues,ques,siamese,sess,three_hash_dict,alpha=0.1)

[0.065969646] 0.0
[0.07350582] 0.21081851067789195
[0.06668651] 0.0
[0.10774422] 0.5222329678670935
[0.09777391] 0.3651483716701107
[0.04108715] 0.24806946917841693
[0.057490468] 0.0
[0.06376827] 0.0
[0.06710744] 0.29814239699997197
[0.058106244] 0.24806946917841693
[0.09434676] 0.4472135954999579
[0.08108848] 0.0
[0.04227364] 0.0
[0.07564771] 0.0
[0.070854545] 0.0
[0.05643195] 0.26967994498529685
[0.07441187] 0.5163977794943223
[0.076114535] 0.0
[0.049179673] 0.5443310539518174
[0.0751034] 0.26967994498529685
[0.09462625] 0.26967994498529685
[0.060709536] 0.3651483716701107
[0.07563734] 0.4364357804719848
[0.055894673] 0.21693045781865616
[0.05292499] 0.0
[0.08568245] 0.4216370213557839
[0.039692104] 0.3999999999999999
[0.053568065] 0.0
[0.06931257] 0.0
[0.05138862] 0.28284271247461895
[0.07618058] 0.0
[0.06558061] 0.0
[0.071058095] 0.2581988897471611
[0.055615842] 0.0
[0.0876829] 0.2581988897471611
[0.07597017] 0.0
[0.081896484] 0.4364357804719848
[0.074851096] 0.3481553119113957
[0.

In [108]:
trainQues[310]

'what is your favorite quotation from Henry David Thoreau ?'

In [125]:
print(score)

(array(['what is a double layer DVD burner?', 'What is a blog?',
       'What is a motherboard?'], dtype=object), [0.906596964597702, 0.7176139222742814, 0.9066686511039734, 0.4407647506115104, 0.5811438564537791, 0.6808461927967979, 0.9057490468025208, 0.9063768267631531, 0.638382586604139, 0.682548102100463, 0.506942440220387, 0.9081088483333588, 0.904227364063263, 0.9075647711753846, 0.9070854544639587, 0.6629312444270969, 0.44268318536001716, 0.9076114535331726, 0.4150200187628529, 0.6647983897270084, 0.6667506743015171, 0.5774374191044596, 0.5147715316297792, 0.710352055250273, 0.9052924990653992, 0.5290949259522952, 0.5439692103862764, 0.9053568065166474, 0.9069312572479248, 0.6505804209058692, 0.9076180577278138, 0.9065580606460572, 0.6747268086777045, 0.9055615842342377, 0.6763892895088508, 0.9075970172882081, 0.51539744596503, 0.5941453288473861, 0.6385130075273721, 0.904669862985611, 0.9067753493785858, 0.46740694458470294, 0.6986404858477773, 0.5034107279267622, 0.5044888269

In [0]:
# prepare data and tf.session
# mnist = input_data.read_data_sets('MNIST_data', one_hot=False)
tf.reset_default_graph()
sess = tf.InteractiveSession() 
vectorizer = CountVectorizer()
tokenizer=vectorizer.build_tokenizer()
rowcount = 0
qList,aList = [],[]
for row in data[0:100]:
  rowcount += 1
  quesVec,ansVec = [0]*64881,[0]*64881
  quesHash = getThreeHash(row[1].lower())
  ansHash = getThreeHash(row[3].lower())
#   print(quesHash)
  for token in tokenizer(quesHash.lower()):
     quesVec[three_hash_dict[token]] += 1
  for token in tokenizer(ansHash.lower()):
     ansVec[three_hash_dict[token]] += 1
  qList.append(quesVec)
  aList.append(ansVec)
  if rowcount % 10 == 0:    
    trainSiamese(qList,aList)
    qList,aList = [],[]
sess.close()
    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py:1702: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


(?, 1947)
(?, 1947)
step 0: loss 9.297
step 10: loss 0.676
step 20: loss 0.385
step 30: loss 0.253
step 40: loss 0.169
step 50: loss 0.113
step 60: loss 0.075
step 70: loss 0.050
step 80: loss 0.034
step 90: loss 0.022


ValueError: ignored